## Example of API's use : upload assets

Prerequisite : The user should have created a project

#### Authenticate via the `signIn` resolver, in order to retrieve the authentication token, then retrieve the token and inject it in the GraphQL client

In [204]:
from graphqlclient import GraphQLClient
import json

#"https://cloud.kili-technology.com/api/label/graphql"
graphql_client = GraphQLClient('http://localhost:4000/graphql')

sign_in = json.loads(graphql_client.execute('''mutation {
    signIn(
        email: "fx@kili-technology.com",
        password: "abc123"
    ) {
        id
        token
        user {
            email
            id
        }
    }
}'''))
token = sign_in['data']['signIn']['token']
graphql_client.inject_token(f'Bearer: {token}')
userID = sign_in['data']['signIn']['user']["id"]
userID

'fourth-user'

#### Prepare the different assets you want to inject

In [93]:
projects = json.loads(graphql_client.execute('''
    query {{
	getProjects(
		userID: "{}"
	)
	{{
		id
	}}
}}
'''.format(userID)))
last_projectID = projects["data"]["getProjects"][-1]["id"]
projects

{'data': {'getProjects': [{'id': 'first-project'},
   {'id': 'second-project'},
   {'id': 'third-project'}]}}

#### Launch the upload assets mutations

In [175]:
import os
path_to_data_directory = "../../Open_Dataset/data/inputs_example/"
extension_type = ".mp3"

for file in os.listdir(path_to_data_directory):
    if extension_type in file:
        content = file
        project_ID = last_projectID
        file_name = file.split(extension_type)[0]
        print(graphql_client.execute('''
                                mutation {{
                                appendToDataset(
                                    projectID:"{}"
                                    content: "{}"
                                    filename: "{}"
                                    isInstructions: false
                                    isHoneypot: true
                                    externalID: "{}"
                                    instructions: ""
                                    consensusMark: 0
                                    honeypotMark: 0
                                    status:TODO
                                )
                                {{
                                    id
                                    dataset{{id}}
                                }}
                            }}
                            '''.format(project_ID, content, file_name, file_name)))


{"data":{"appendToDataset":{"id":"third-project","dataset":[{"id":"cjxltdu7o08zt0798h7dc4xhw"},{"id":"cjxltdu7s08zv0798q3owl5yx"},{"id":"cjxltdu7t08zw079851jf61pb"},{"id":"cjxltdu7u08zx0798smegavtl"},{"id":"cjxltea0509220798fu8nrm7v"},{"id":"cjxltea1m092e07980l0bt54o"},{"id":"cjxltf3je094h0798srjq3tsy"},{"id":"cjxltf3ko094t0798e90jwi0k"},{"id":"cjxltfs3g09550798uxf8cgdj"},{"id":"cjxltfs4t095h079856c6slhj"},{"id":"cjxltyq7y09600798otfdp1op"}]}}}
{"data":{"appendToDataset":{"id":"third-project","dataset":[{"id":"cjxltdu7o08zt0798h7dc4xhw"},{"id":"cjxltdu7s08zv0798q3owl5yx"},{"id":"cjxltdu7t08zw079851jf61pb"},{"id":"cjxltdu7u08zx0798smegavtl"},{"id":"cjxltea0509220798fu8nrm7v"},{"id":"cjxltea1m092e07980l0bt54o"},{"id":"cjxltf3je094h0798srjq3tsy"},{"id":"cjxltf3ko094t0798e90jwi0k"},{"id":"cjxltfs3g09550798uxf8cgdj"},{"id":"cjxltfs4t095h079856c6slhj"},{"id":"cjxltyq7y09600798otfdp1op"},{"id":"cjxltyq9w096d0798hk1a59px"}]}}}


#### Launch the download of assets

In [192]:
graphql_client.execute('''query {getAssets(projectID: "third-project", skip: 0, first: 20){id}}''')

'{"data":{"getAssets":[{"id":"cjxlufi0409ca07985oop0w4x"},{"id":"cjxlufi0709cc0798ykb74r5t"},{"id":"cjxlufi0809cd0798a72dxurb"},{"id":"cjxlufi0809ce0798qe49keyk"}]}}'

In [193]:
graphql_client.execute('''
    query {getAsset(assetID: "cjxlufi0809ce0798qe49keyk" )
    {
    id
    content
    }
}''')

'{"data":{"getAsset":{"id":"cjxlufi0809ce0798qe49keyk","content":"https://cdn.pixabay.com/photo/2018/03/10/23/27/fruit-3215625_1280.jpg"}}}'

#### Upload labels of assets

In [205]:
graphql_client.execute('''
    mutation {appendToLabels(
    authorID: "fourth-user"
    isReview: false
    jsonResponse: """{\"annotations\":[{\"id\":\"5debfe82-3ba9-4727-8ddf-bc256b6ab050\",\"score\":null,\"description\":\"GRAPE\",\"isCut\":false,\"boundingPoly\":[{\"vertices\":[{\"x\":10.412573673870334,\"y\":16.8141592920354},{\"x\":28.68369351669941,\"y\":16.8141592920354},{\"x\":28.68369351669941,\"y\":63.421828908554566},{\"x\":10.412573673870334,\"y\":63.421828908554566}]}]}]}"""
    labelType: IMAGE
    millisecondsToLabel: 0
    labelAssetID: "cjxlufi0809ce0798qe49keyk"
  )
  {
  id
  
  }
}''')

'{"data":{"appendToLabels":{"id":"third-project"}}}'